# tools.py 디버깅 테스트

tool_turn_resolution 함수를 테스트합니다.

## 1. 환경 설정

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트를 path에 추가
root = Path.cwd()
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print(f"프로젝트 루트: {root}")

In [ ]:
import os
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG, format="%(levelname)s %(name)s %(message)s")

## 2. 시나리오 로드

In [ ]:
from app.loader import ScenarioLoader

base_path = root / "scenarios"
loader = ScenarioLoader(base_path)
scenario_ids = loader.list_scenarios()

print(f"사용 가능한 시나리오: {scenario_ids}")

if not scenario_ids:
    raise Exception("시나리오가 없습니다!")

scenario_id = scenario_ids[0]
assets = loader.load(scenario_id)
print(f"\n로드된 시나리오: {scenario_id}")
print(f"  - 제목: {assets.scenario.get('title')}")
print(f"  - NPC: {assets.get_all_npc_ids()}")
print(f"  - 아이템: {assets.get_all_item_ids()}")

## 3. 월드 상태 생성

In [ ]:
from app.models import WorldState, NPCState

world = WorldState(
    turn=1,
    npcs={
        "family": NPCState(npc_id="family", trust=0, fear=0, suspicion=0),
        "partner": NPCState(npc_id="partner", trust=0, fear=0, suspicion=1),
        "witness": NPCState(npc_id="witness", trust=0, fear=2, suspicion=0),
    },
    inventory=["casefile_brief", "pattern_analyzer"],
    vars={"clue_count": 0, "fabrication_score": 0},
)

print(f"WorldState:")
print(f"  - turn: {world.turn}")
print(f"  - npcs: {list(world.npcs.keys())}")
print(f"  - inventory: {world.inventory}")
print(f"  - vars: {world.vars}")

## 4. LLM 로드

In [ ]:
from app.llm import LLM_Engine

model_name = os.environ.get("LLM_MODEL", "Qwen/Qwen3-8B")
print(f"LLM 로딩 중: {model_name}")
print("(환경변수 LLM_MODEL로 변경 가능)")

llm = LLM_Engine(model_name=model_name)
print("LLM 로드 완료!")

## 5. 프롬프트 확인

In [ ]:
from app.llm import build_prompt

user_input = "피해자 가족에게 그날 있었던 일을 물어본다"

prompt = build_prompt(
    user_input=user_input,
    world_state=world.to_dict(),
    memory_summary=None,
    npc_context=assets.export_for_prompt(),
)

print(f"프롬프트 길이: {len(prompt)} 자")
print("=" * 60)
print(prompt)
print("=" * 60)

## 6. tool_turn_resolution 테스트

In [ ]:
from app.tools import tool_turn_resolution

user_input = "피해자 가족에게 그날 있었던 일을 물어본다"

print(f"입력: {user_input}")
print("LLM 생성 중...")

result = tool_turn_resolution(user_input, world, assets, llm)

print("\n결과:")
print(f"  event_description: {result.event_description}")
print(f"  state_delta: {result.state_delta}")

## 7. 여러 테스트 케이스

In [ ]:
test_cases = [
    "피해자 가족에게 그날 있었던 일을 물어본다",
    "패턴 분석기를 사용한다",
    "현장을 조사한다",
]

print(f"테스트 케이스 ({len(test_cases)}개)")
print("=" * 60)

for i, text in enumerate(test_cases, 1):
    print(f"\n[{i}] 입력: \"{text}\"")
    result = tool_turn_resolution(text, world, assets, llm)
    print(f"    사건: {result.event_description}")
    print(f"    델타: {result.state_delta}")

print("\n" + "=" * 60)
print("테스트 완료!")

## 8. Raw LLM 출력 확인

In [ ]:
from app.llm import parse_response

user_input = "목격자에게 범인의 인상착의를 물어본다"

prompt = build_prompt(
    user_input=user_input,
    world_state=world.to_dict(),
    memory_summary=None,
    npc_context=assets.export_for_prompt(),
)

print(f"입력: {user_input}")
print("\nLLM 생성 중...")

raw_output = llm.generate(prompt)

print("\n[Raw LLM Output]")
print("-" * 40)
print(raw_output)
print("-" * 40)

print("\n[Parsed Response]")
parsed = parse_response(raw_output)
print(f"  state_delta: {parsed.state_delta}")
print(f"  event_description: {parsed.event_description}")